In [6]:
import pandas as pd
import glob

# Define seasons and the list to hold training data
seasons = ['2021-22', '2022-23', '2023-24']
train_data = []

# Iterate through each season and its respective gameweeks
for season in seasons:
    for gw in range(1, 39):
        try:
            # Load current gameweek data
            current_gw_file = f"../gw_data/{season}/gw{gw}.csv"  # Adjust the path as necessary
            current_gw_df = pd.read_csv(current_gw_file)

            # Select relevant fields
            current_gw_df = current_gw_df[[
                'name', 'position', 'team', 'assists', 'bonus', 'bps', 
                'clean_sheets', 'creativity', 'element', 'fixture', 
                'goals_conceded', 'goals_scored', 'ict_index', 
                'influence', 'minutes', 'own_goals', 
                'penalties_missed', 'penalties_saved', 'red_cards', 
                'saves', 'team_a_score', 'team_h_score', 'threat', 
                'was_home', 'yellow_cards'
            ]]
            
            # Add season and gameweek columns
            current_gw_df['season'] = season
            current_gw_df['gameweek'] = gw
            
            # Identify next gameweek
            next_gw = gw + 1

            # Check if next gameweek exists in the current season or the next season
            if next_gw <= 38:
                next_gw_file = f"../gw_data/{season}/gw{next_gw}.csv"
            elif next_gw == 39:  # If current gw is 38, link to next season's gw1
                next_season = seasons[seasons.index(season) + 1]
                next_gw_file = f"../gw_data/{next_season}/gw1.csv"
            else:
                continue  # Skip if no valid next gameweek

            # Load next gameweek data
            next_gw_df = pd.read_csv(next_gw_file)

            # Merge to get total points from the next gameweek
            current_gw_df = current_gw_df.merge(
                next_gw_df[['name', 'total_points']],
                on='name',
                how='left'
            )

            # Rename the 'total_points' column
            current_gw_df.rename(columns={'total_points': 'next_gw_points'}, inplace=True)

            # Append to training data list
            train_data.append(current_gw_df)

        except FileNotFoundError:
            print(f"File for {season} Gameweek {gw} not found. Skipping...")
        except Exception as e:
            print(f"An error occurred while processing {season} Gameweek {gw}: {e}")

# Combine all data into a single DataFrame
train_data_df = pd.concat(train_data, ignore_index=True)

# Sort by next gameweek points in descending order
train_data_df = train_data_df.sort_values(by='next_gw_points', ascending=False)

# Optionally, save the training data to a CSV
train_data_df.to_csv('train_data.csv', index=False)

print("Training data created successfully.")


An error occurred while processing 2023-24 Gameweek 38: list index out of range


/var/folders/vz/717hjk_x7gd2h1zp14m7dtcm0000gn/T/ipykernel_72429/3462279436.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_data_df = pd.concat(train_data, ignore_index=True)


Training data created successfully.
